In [16]:
import pandas as pd     

df = pd.read_csv('glassdoor_jobs.csv')
pd.set_option("display.max_columns", 12)
pd.set_option("display.max_rows", 15)

In [17]:
#Salary Parsing
df['hourly'] = df["Salary Estimate"].apply(lambda x: 1 if "per hour" in x.lower() else 0)
df['employer_provided'] = df["Salary Estimate"].apply(lambda x: 1 if 'employer provided salary:' in x.lower() else 0)

In [18]:
df = df[df["Salary Estimate"] != '-1']
salary = df["Salary Estimate"].apply(lambda x: x.split('(')[0])
minus_kd = salary.apply(lambda x: x.replace('K', '').replace("$",""))

In [19]:
min_hr = minus_kd.apply(lambda x: x.lower().replace('per hour','').replace('employer provided salary:','')) 

In [20]:
df[["min_salary", "max_salary"]] = min_hr.str.split("-").apply(pd.Series).ffill(axis=1)
df['min_salary'] = pd.to_numeric(df['min_salary'])
df['max_salary'] = pd.to_numeric(df['max_salary'])


In [21]:
df['avg_salary'] = (df.max_salary+df.min_salary)/2

In [22]:
#Grab just the company name
df['company_name'] = df.apply(lambda x: x['Company Name'] if x["Rating"] < 0 else x["Company Name"][:-3], axis =1)
df['company_name'] = df.company_name.apply(lambda x: x.replace('\n', ''))

In [23]:
#Grab the State from Location
#Looks like all NaN values are Remote jobs so i will fill with that.
#Also looks like a county in Ohio appers as the State so ill fix that as well
df['job_state'] = df['Location'].str.split(',').str[1]
df["job_state"].replace({"nan": "Remote", " Cuyahoga": "OH"}, inplace=True)
df['job_state'] = df['job_state'].fillna("Remote")

In [24]:
df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Size', 'Founded', 'Type of ownership',
       'Industry', 'Sector', 'Revenue', 'hourly', 'employer_provided',
       'min_salary', 'max_salary', 'avg_salary', 'company_name', 'job_state'],
      dtype='object')

In [25]:
#parsing job description for skills needed 
df['python'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
df['excel'] = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)
df['sql'] = df['Job Description'].apply(lambda x: 1 if 'sql' in x.lower() else 0)



In [ ]:
df.to_csv('clean_salary_data.csv', index= False)